# BTS concert twt data depersonalisation 
The notebook transforming the twitter API collected tweet dataset into an anonimized version still carrying the info used to look at tweet timing during live online broacast BTS concerts in 2021.

Info to retain:

	- Twt timing
    - Twt ID # replaced
	- Account ID # replaced
	- RTed Twt ID # replaced (unique)
    - RTing following count (for impression estimate)
	- Likes accumulated 
	- Retweets accumulated
	- Tweet length
	- Media inclusion (QT, photo, video, website preview)

Accourding to API research agreement, can not release full data sets. Two sets of reductions

Tweets for content analysis (1200, not all can be expected to be available). If a tweet exists find it with construction: http://.... TweetID

	tweet ID (real) with minimum url (tweet ID not user name)
    codes assigned
    User stats, RTing and Original (wait is this relevant? reported in summary in one table.)
    Tweet length
    If media embeded (links, media hosted on twitter or QT)
    Concert associated
    Which subset 
    

	

In [1]:
import sys
import os
import time
import datetime as dt
import math
import numpy as np 
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy.signal import butter,filtfilt
from scipy import interpolate
from scipy.interpolate import interp1d

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

/Users/finn/.pyenv/versions/3.9.0/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [3]:
# import respy functions from twt.py file
%load_ext autoreload
%autoreload 1
%aimport twt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
data_dir = '/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/'

In [8]:
Concerts = pd.DataFrame(columns=['tag','data_loc','twt_dbs','event_file','event_offset','event_reduction','Long_name','sampling','Program'])
Concerts.loc[0,:]={'tag': 'SWZ_D1','data_loc':'/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/',
             'twt_dbs':'Fan_tweets_H_Sowoozoo_D1.csv','event_file':'Setlists_sowoozoo_D1.csv','event_offset':'6MIN','event_reduction':[1,2,3,6,8,9,10,12,13,15,16,19,20,21,22,23,25,26,27,28],
             'Long_name':'Sowoozoo Concert Day 1','sampling':'#SOWOOZOO','Program':'SWZ'}
Concerts.loc[1,:]={'tag': 'SWZ_D2','data_loc':'/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/',
             'twt_dbs':'Fan_tweets_H_Sowoozoo_D2.csv','event_file':'Setlists_sowoozoo_D2.csv','event_offset':'108S','event_reduction':[1,2,3,6,8,9,10,12,13,15,16,19,20,21,22,23,25,26,27,28],
             'Long_name':'Sowoozoo Concert Day 2','sampling':'#SOWOOZOO','Program':'SWZ'}
Concerts.loc[2,:]={'tag': 'PTD_ON','data_loc':'/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/PTD/',
             'twt_dbs':'FullPTD_Fan_tweets_PTD_ON_STAGE.csv','event_file':'Setlists_PTD_ON.csv','event_offset':'25S','event_reduction':[1,2,3,6,7,8,9,11,12,14,15,17,18,21,22,28,29,32,33,34,36,37,38,39],
             'Long_name':'Permission to Dance on Stage','sampling':'Kpop Stream','Program':'PTD'}
Concerts.loc[3,:]={'tag': 'PTD_LA4','data_loc':'/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/PTD/',
             'twt_dbs':'PTD_LA4_Fan_tweets_FULLSTREAM.csv','event_file':'Setlists_PTD_LA4.csv','event_offset':'40S','event_reduction':[1,2,3,6,7,8,9,11,12,14,15,17,18,21,22,28,29,32,33,34,36,37,38,39],
             'Long_name':'Permission to Dance LA Day 4','sampling':'Kpop Stream','Program':'PTD'}
Concerts.loc[4,:]={'tag': 'PTD_ON_Alt1','data_loc':'/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/PTD/',
             'twt_dbs':'Alt1PTD_Fan_tweets_PTD_ON_STAGE.csv','event_file':'','event_offset':'','event_reduction':[],
             'Long_name':'Week prior to PTD On Stage','sampling':'Kpop Stream','Program':''}
Concerts.loc[5,:]={'tag': 'PTD_ON_Alt2','data_loc':'/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/PTD/',
             'twt_dbs':'Alt2PTD_Fan_tweets_PTD_ON_STAGE.csv','event_file':'','event_offset':'','event_reduction':[],
             'Long_name':'Week following PTD On Stage','sampling':'Kpop Stream','Program':''}
Concerts

,tag,data_loc,twt_dbs,event_file,event_offset,event_reduction,Long_name,sampling,Program
0,SWZ_D1,/Users/finn/Desktop/Current_Projects/BTS_twitt...,Fan_tweets_H_Sowoozoo_D1.csv,Setlists_sowoozoo_D1.csv,6MIN,"[1, 2, 3, 6, 8, 9, 10, 12, 13, 15, 16, 19, 20,...",Sowoozoo Concert Day 1,#SOWOOZOO,SWZ
1,SWZ_D2,/Users/finn/Desktop/Current_Projects/BTS_twitt...,Fan_tweets_H_Sowoozoo_D2.csv,Setlists_sowoozoo_D2.csv,108S,"[1, 2, 3, 6, 8, 9, 10, 12, 13, 15, 16, 19, 20,...",Sowoozoo Concert Day 2,#SOWOOZOO,SWZ
2,PTD_ON,/Users/finn/Desktop/Current_Projects/BTS_twitt...,FullPTD_Fan_tweets_PTD_ON_STAGE.csv,Setlists_PTD_ON.csv,25S,"[1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 17, 18, ...",Permission to Dance on Stage,Kpop Stream,PTD
3,PTD_LA4,/Users/finn/Desktop/Current_Projects/BTS_twitt...,PTD_LA4_Fan_tweets_FULLSTREAM.csv,Setlists_PTD_LA4.csv,40S,"[1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 17, 18, ...",Permission to Dance LA Day 4,Kpop Stream,PTD
4,PTD_ON_Alt1,/Users/finn/Desktop/Current_Projects/BTS_twitt...,Alt1PTD_Fan_tweets_PTD_ON_STAGE.csv,,,[],Week prior to PTD On Stage,Kpop Stream,
5,PTD_ON_Alt2,/Users/finn/Desktop/Current_Projects/BTS_twitt...,Alt2PTD_Fan_tweets_PTD_ON_STAGE.csv,,,[],Week following PTD On Stage,Kpop Stream,


In [40]:
df_alltwt=pd.read_csv(data_dir +'Fan_tweets_H_Sowoozoo_D1.csv')
data_name = 'SWZ_D1'

In [10]:
i = 1
df_alltwt=pd.read_csv(Concerts.loc[i,'data_loc'] + Concerts.loc[i,'twt_dbs'])
data_name = Concerts.loc[i,'tag']

/Users/finn/.pyenv/versions/3.9.0/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [132]:
filtered_users = ['@BTS_twt','@bts_bighit','@weverseofficial','@weverseshop','@BIGHIT_MUSIC','@HYBE_MERCH','@BT21_']
dontuse_users = [          426850435,          4822956857,          3355353969,
       1315377647678099456, 1309207635908898819,  728053993335742464,
       1214462573845696513, 1311326608792272901, 1402927845899264003,
       1220206617158770689, 1245616096809324547, 1288561623670517760]
# optional to remove official accounts engagements
print('Size of full set: ' + str(len(df_alltwt)))
df_fantwt = df_alltwt.copy()
for acc in filtered_users:
    twts = df_fantwt['tweet'] 
    df_fantwt = df_fantwt.loc[~(twts.str.contains('T '+acc+':', case=False,regex=False))].copy()

for acc in filtered_users:
    twts = df_fantwt['user_screen_name'] 
    df_fantwt = df_fantwt.loc[~(twts.str.contains(acc[1:], case=False,regex=False))].copy()

for acc in dontuse_users:
    df_fantwt = df_fantwt.loc[df_fantwt['user_id']!=acc,:].copy()

print('Size of cleaned set: ' + str(len(df_fantwt)))
df_alltwt = df_fantwt.copy()

Size of full set: 225993
Size of cleaned set: 224947


In [12]:
df_alltwt["created_at"] = pd.to_datetime(df_alltwt["created_at"])
df_alltwt_ord = df_alltwt.sort_values('created_at')
df_alltwt = df_alltwt_ord.reset_index(drop=True)

In [13]:
print(df_alltwt.columns)
# retain 'created_at', 'user_followers_count','retweeted_status_id',
# 'retweeted_status_retweet_count', 'retweeted_status_favorite_count','retweeted_status_reply_count',
Feilds_to_keep = ['created_at', 'user_id','user_followers_count','retweeted_status_id','retweeted_status_retweet_count', 'retweeted_status_favorite_count','retweeted_status_reply_count',]
df_Reduced = df_alltwt.loc[:,Feilds_to_keep].copy()


Index(['id', 'created_at', 'tweet', 'source', 'language', 'user_id',
       'user_screen_name', 'user_name', 'user_description', 'user_language',
       'user_location', 'user_created_at', 'user_followers_count',
       'user_friends_count', 'user_statuses_count', 'user_favorites_count',
       'user_verified', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_user_screen_name', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_user_screen_name',
       'retweeted_status_user_name', 'retweeted_status_user_description',
       'retweeted_status_user_friends_count',
       'retweeted_status_user_statuses_count',
       'retweeted_status_user_followers_count',
       'retweeted_status_retweet_count', 'retweeted_status_favorite_count',
       'retweeted_status_reply_count', 'quoted_status_id',
       'quoted_status_user_id', 'quoted_status_user_screen_name',
       'quoted_status_user_name', 'quoted_status_user_description',
       'quoted_status

In [14]:
Feilds_to_anon = ['user_id','retweeted_status_id',]
for replace_feild in Feilds_to_anon:
    A = df_alltwt[replace_feild].value_counts().reset_index()
    B = dict((v, k) for k, v in A['index'].to_dict().items())
    df_Reduced[replace_feild].replace(B,inplace = True)

In [15]:
# add in binary: RT, QT, Reply, Original
df_Reduced['Original'] = 1
df_Reduced.loc[df_alltwt['retweeted_status_id'].notna(),'Original'] = 0
df_Reduced.loc[df_alltwt['quoted_status_id'].notna(),'Original'] = 0
df_Reduced.loc[df_alltwt['in_reply_to_status_id'].notna(),'Original'] = 0
df_Reduced['RT'] = 0
df_Reduced.loc[df_alltwt['retweeted_status_id'].notna(),'RT'] = 1
df_Reduced['QT'] = 0
df_Reduced.loc[df_alltwt['quoted_status_id'].notna(),'QT'] = 1
df_Reduced['Reply'] = 0
df_Reduced.loc[df_alltwt['in_reply_to_status_id'].notna(),'Reply'] = 1

twts = df_alltwt['tweet']
df_Reduced['Media'] = 0
df_Reduced.loc[twts.str.contains('https://t.co', case=False,regex=False),'Media'] = 1
df_Reduced['Length'] = twts.str.len()
df_Reduced.iloc[:5,:]

,created_at,user_id,user_followers_count,retweeted_status_id,retweeted_status_retweet_count,retweeted_status_favorite_count,retweeted_status_reply_count,Original,RT,QT,Reply,Media,Length
0,2021-06-14 09:00:00+00:00,2868,23,104.0,260.0,133.0,0.0,0,1,0,0,1,235
1,2021-06-14 09:00:00+00:00,39131,39,6.0,4107.0,13410.0,66.0,0,1,0,0,1,86
2,2021-06-14 09:00:01+00:00,9742,2,1873.0,820.0,3352.0,15.0,0,1,0,0,1,59
3,2021-06-14 09:00:01+00:00,29607,1378,NaN,NaN,NaN,NaN,1,0,0,0,0,66
4,2021-06-14 09:00:01+00:00,36272,71,NaN,NaN,NaN,NaN,1,0,0,0,0,54


In [16]:
df_Reduced.to_csv('./data/'+data_name+'_reduced.csv')

In [248]:
filtered_users = ['@BTS_twt','@bts_bighit','@weverseofficial','@weverseshop','@BIGHIT_MUSIC','@HYBE_MERCH','@BT21_']
dontuse_users = [ 3890555843,         426850435,          4822956857,          3355353969,
       1315377647678099456, 1309207635908898819,  728053993335742464,
       1214462573845696513, 1311326608792272901, 1402927845899264003,
       1220206617158770689, 1245616096809324547, 1288561623670517760]

for i in Concerts.index:
    df_alltwt=pd.read_csv(Concerts.loc[i,'data_loc'] + Concerts.loc[i,'twt_dbs'])
    data_name = Concerts.loc[i,'tag']
    print(data_name)
    # Clean up
    
    # optional to remove official accounts engagements
    print('Size of full set: ' + str(len(df_alltwt)))
    df_fantwt = df_alltwt.copy()
    for acc in filtered_users:
        twts = df_fantwt['tweet'] 
        df_fantwt = df_fantwt.loc[~(twts.str.contains('T '+acc+':', case=False,regex=False))].copy()
        
    for acc in filtered_users:
        twts = df_fantwt['user_screen_name'] 
        df_fantwt = df_fantwt.loc[~(twts.str.contains(acc[1:], case=False,regex=False))].copy()

    for acc in dontuse_users:
        df_fantwt = df_fantwt.loc[df_fantwt['user_id']!=acc,:].copy()

    df_notreplys = df_fantwt.loc[df_fantwt['in_reply_to_user_id'].isna(),:]
    df_replys = df_fantwt.loc[df_fantwt['in_reply_to_user_id'].notna(),:]
    print('Size of reply tweets: ' + str(len(df_replys)) )
    twts = df_replys['in_reply_to_user_screen_name']  # df_replys['in_reply_to_user_screen_name'].value_counts()
    for acc in filtered_users: # filter RTs and replys
         df_replys =  df_replys.loc[~(twts.str.startswith(acc[1:]))]
    print('Size of replys: ' + str(len(df_replys)) )
    df_alltwt = pd.concat([df_notreplys,df_replys])

    print('Size of cleaned set: ' + str(len(df_alltwt)))
    
    df_alltwt["created_at"] = pd.to_datetime(df_alltwt["created_at"])
    df_alltwt = df_alltwt.sort_values('created_at').reset_index(drop=True)

    Feilds_to_keep = ['created_at','id','user_id','user_followers_count','retweeted_status_id','retweeted_status_user_id',
                      'retweeted_status_retweet_count', 'retweeted_status_favorite_count','retweeted_status_reply_count',
                      'quoted_status_id','quoted_status_user_id','in_reply_to_status_id','in_reply_to_user_id' ]
    df_Reduced = df_alltwt.loc[:,Feilds_to_keep].copy()

    # Anon status Id :'id','in_reply_to_status_id', 'retweeted_status_id', 'quoted_status_id',
    Feilds_to_anon = ['id','in_reply_to_status_id', 'retweeted_status_id', 'quoted_status_id']
    ids = []
    for replace_feild in Feilds_to_anon:
        ids +=list(df_alltwt.loc[df_alltwt[replace_feild].notna(),replace_feild].astype('int64').values)
    ids=pd.DataFrame(columns = ['index'],data = ids)
    A = ids.value_counts().reset_index()
    B = dict((v, k) for k, v in A['index'].to_dict().items())
    tic = time.time()
    for replace_feild in Feilds_to_anon:
        df_Reduced[replace_feild].replace(B,inplace = True)
        print([replace_feild,time.time()-tic])
        
    # Anon user ids : 'user_id','in_reply_to_user_id','retweeted_status_user_id','quoted_status_user_id',
    Feilds_to_anon = ['user_id','in_reply_to_user_id','retweeted_status_user_id','quoted_status_user_id']
    ids = []
    for replace_feild in Feilds_to_anon:
        ids +=list(df_alltwt.loc[df_alltwt[replace_feild].notna(),replace_feild].astype('int64').values)
    ids=pd.DataFrame(columns = ['index'],data = ids)
    A = ids.value_counts().reset_index()
    B = dict((v, k) for k, v in A['index'].to_dict().items())
    tic = time.time()
    for replace_feild in Feilds_to_anon:
        df_Reduced[replace_feild].replace(B,inplace = True)
        print([replace_feild,time.time()-tic])
    

    # add in binary: RT, QT, Reply, Original
    df_Reduced['Original'] = 1
    df_Reduced.loc[df_alltwt['retweeted_status_id'].notna(),'Original'] = 0
    df_Reduced.loc[df_alltwt['quoted_status_id'].notna(),'Original'] = 0
    df_Reduced.loc[df_alltwt['in_reply_to_status_id'].notna(),'Original'] = 0
    df_Reduced['RT'] = 0
    df_Reduced.loc[df_alltwt['retweeted_status_id'].notna(),'RT'] = 1
    df_Reduced['QT'] = 0
    df_Reduced.loc[df_alltwt['quoted_status_id'].notna(),'QT'] = 1
    df_Reduced['Reply'] = 0
    df_Reduced.loc[df_alltwt['in_reply_to_status_id'].notna(),'Reply'] = 1

    twts = df_alltwt['tweet']
    df_Reduced['Media'] = 0
    df_Reduced.loc[twts.str.contains('https://t.co', case=False,regex=False),'Media'] = 1
    df_Reduced['Length'] = twts.str.len()
    
    df_Reduced.to_csv('./data/'+data_name+'_reduced.csv')
    print(data_name)

SWZ_D1
Size of full set: 225993
Size of reply tweets: 1730
Size of replys: 1484
Size of cleaned set: 224700
['id', 142.29803705215454]
['in_reply_to_status_id', 244.81995296478271]
['retweeted_status_id', 349.37770414352417]
['quoted_status_id', 453.4455392360687]
['user_id', 61.579460859298706]
['in_reply_to_user_id', 107.51586103439331]
['retweeted_status_user_id', 153.37042093276978]
['quoted_status_user_id', 199.41680693626404]
SWZ_D1


/Users/finn/.pyenv/versions/3.9.0/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (33,34,35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


SWZ_D2
Size of full set: 114728
Size of reply tweets: 1303
Size of replys: 947
Size of cleaned set: 111153
['id', 33.579432010650635]
['in_reply_to_status_id', 58.25239300727844]
['retweeted_status_id', 82.93545293807983]
['quoted_status_id', 107.78708100318909]
['user_id', 11.886307001113892]
['in_reply_to_user_id', 20.599513053894043]
['retweeted_status_user_id', 29.210964918136597]
['quoted_status_user_id', 37.66882109642029]
SWZ_D2
PTD_ON
Size of full set: 277925
Size of reply tweets: 56674
Size of replys: 8213
Size of cleaned set: 228732
['id', 139.27385210990906]
['in_reply_to_status_id', 247.17062091827393]
['retweeted_status_id', 356.0638470649719]
['quoted_status_id', 463.31788897514343]
['user_id', 45.74487495422363]
['in_reply_to_user_id', 79.92533206939697]
['retweeted_status_user_id', 116.13795399665833]
['quoted_status_user_id', 150.15946984291077]
PTD_ON
PTD_LA4
Size of full set: 143837
Size of reply tweets: 28387
Size of replys: 4069
Size of cleaned set: 116331
['id', 3

In [247]:
df_Reduced

,created_at,id,user_id,user_followers_count,retweeted_status_id,retweeted_status_user_id,retweeted_status_retweet_count,retweeted_status_favorite_count,retweeted_status_reply_count,quoted_status_id,quoted_status_user_id,in_reply_to_status_id,in_reply_to_user_id
0,2021-06-13 09:00:00+00:00,155042,1327624878975926273,16,48.0,1.350775e+18,507.0,105.0,1.0,NaN,NaN,NaN,NaN
1,2021-06-13 09:00:00+00:00,154882,1170195124443201536,4,465.0,1.393068e+18,20.0,20.0,0.0,NaN,NaN,NaN,NaN
2,2021-06-13 09:00:00+00:00,154881,1155841633755119616,24,23.0,1.402049e+18,34.0,24.0,0.0,NaN,NaN,NaN,NaN
3,2021-06-13 09:00:00+00:00,154961,1402666111577952258,0,19.0,1.225861e+18,582.0,67.0,2.0,NaN,NaN,NaN,NaN
4,2021-06-13 09:00:00+00:00,155044,29280045,264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224695,2021-06-13 12:29:59+00:00,79031,1253200683639148544,621,206.0,1.264793e+18,175.0,608.0,8.0,NaN,NaN,NaN,NaN
224696,2021-06-13 12:29:59+00:00,231542,1345256812744822784,2580,33.0,5.988054e+07,688.0,2234.0,9.0,NaN,NaN,NaN,NaN
224697,2021-06-13 12:29:59+00:00,79019,1127603431479234564,8967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62027.0,1.127603e+18
224698,2021-06-13 12:29:59+00:00,79008,1407110581,182,77760.0,1.316138e+18,157.0,458.0,1.0,NaN,NaN,NaN,NaN


# Check on coded subsets
Clean up coded subset for easy processing

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html

In [46]:
# subTwts = pd.read_csv(dloc + 'Packed_SWZ_Subsets_Coded.csv')
dloc='/Users/finn/Desktop/Current_Projects/BTS_twitter/twt_Analysis/data/'
# subTwts = pd.read_csv(dloc + 'Packed_PTD_Subsets_Coded_Filtered.csv')
subTwts = pd.read_csv(dloc + 'Packed_PTD_Subsets_Coded.csv')

data_name = subTwts.loc[0,'Concert']
subTwts['Last_RT_created_at'] = pd.to_datetime(subTwts['Last_RT_created_at'])
subTwts['Last_RT_id'] = pd.to_datetime(subTwts['Last_RT_created_at'])
subTwts['Last_RT_id'] = subTwts['Last_RT_id'].astype('int64')
# convert codes to boolean or binary. binary? 
codes = ['Affection', 'Intensifiers', 'Self', 'Members', 'recording', 'Stills',
       'Production', 'Music', 'Commentary', 'Army', 'Anticipation', 'Fanwork',
       'Information', 'Stream', 'Spam', 'Commercial']
for c in codes:
    subTwts.loc[subTwts[c].notna(),c] = 1
    subTwts.loc[subTwts[c].isna(),c] = 0
    subTwts[c] = subTwts[c].astype('bool')

subTwts.columns

Index(['RT_SubSet', 'Concert', 'Last_RT_created_at', 'Last_RT_id',
       'Last_RT_url', 'OriTwt_id', 'language', 'LastRT_user_followers_count',
       'Last_RT_user_friends_count', 'Last_RT_user_statuses_count',
       'Last_RT_user_favorites_count', 'Ori_Twt_user_friends_count',
       'Ori_Twt_user_statuses_count', 'Ori_Twt_user_followers_count',
       'Last_RT_retweet_count', 'Last_RT_favorite_count',
       'Last_RT_reply_count', 'Shout', 'Tweet Length', 'Tweet link/media',
       'Lost', 'Unrelated', 'Affection', 'Intensifiers', 'Self', 'Members',
       'recording', 'Stills', 'Production', 'Music', 'Commentary', 'Army',
       'Anticipation', 'Fanwork', 'Information', 'Stream', 'Spam',
       'Commercial'],
      dtype='object')

In [42]:
# check formating
for col in subTwts.columns:
    N = subTwts[col].nunique()
    print(' '.join([col,'unique',str(N), str(subTwts[col].dtypes)]))
    if N<10:
        print(subTwts[col].unique())
        

RT_SubSet unique 4 object
['Rand200_32t6RTd' 'Rand200_3t1RTd' 'Rand200_NoRT' 'Top200RTd']
Concert unique 1 object
['PTD_ON1']
Last_RT_created_at unique 390 datetime64[ns, UTC]
Last_RT_id unique 390 int64
Last_RT_url unique 400 object
OriTwt_id unique 400 int64
language unique 23 object
LastRT_user_followers_count unique 257 int64
Last_RT_user_friends_count unique 280 int64
Last_RT_user_statuses_count unique 400 int64
Last_RT_user_favorites_count unique 387 int64
Ori_Twt_user_friends_count unique 285 int64
Ori_Twt_user_statuses_count unique 378 int64
Ori_Twt_user_followers_count unique 375 int64
Last_RT_retweet_count unique 129 int64
Last_RT_favorite_count unique 176 int64
Last_RT_reply_count unique 80 int64
Shout unique 2 bool
[False  True]
Tweet Length unique 156 int64
Tweet link/media unique 2 bool
[ True False]
Affection unique 2 bool
[False  True]
Intensifiers unique 2 bool
[ True False]
Self unique 2 bool
[False  True]
Members unique 2 bool
[False  True]
recording unique 2 bool
[F

In [47]:
 subTwts.to_csv('./data/'+data_name+'_coded_subsets_with_replaced.csv')

# check keyword frequencies
Capture countable characteristics of the datasets that are not being retained in the publishable data

Pandemic and covid mentions

Objections to being quoted

The data collected was only of accounts that were public at the time of sampling, and by Twitters terms of service and the Twitter API license used by the university, it was legal to collect these status updates and statistics. However there have been times when twitter users have been very concerned about the way their data is collected and converted into produces for media and research, commercial and academic. 

Out of respect for these concerns, we have taken measures to protect the users whos data we have studied, taking many steps to mitigating risks of harmful visibility via the published datasets and reporting on this research. 

One way in which twitter users have tried to discourage (legal but unethical) citation and data collection is to mark their resistance in their twitter bios. This is a field that we did not use in our research but it is collected by the Twitter API, so we try to find instances in the material we have. 

In [105]:
i = 0
df_alltwt=pd.read_csv(Concerts.loc[i,'data_loc'] + Concerts.loc[i,'twt_dbs'])

df_bios = df_alltwt.loc[df_alltwt['user_description'].notna(),:].copy()
twts = df_bios['user_description'].unique()
print([len(df_alltwt),len(df_bios),len(twts)])
twts[:5]


[225993, 189371, 90454]


array(['I purple you 💜💜',
       '99line/BTS/🐤ペン/推しカプ→→🐰🐥/🦄🐭/🐨🐹/🐯🐣/rps/20↑📛 交換垢→@miichim_koukan',
       'เราจะรวยเมื่อไหร่คะ #อิงอิงจะเเจก',
       'Association BTS France 🇫🇷 Fanbase française dédiée à @BTS_twt et au fandom ARMY 🐋💜 Back Up @ProjetBtsFrance | Site http://www.btsfrance.fr/',
       '❥\n𝐼𝑛𝑠𝑡𝑎𝑔𝑟𝑎𝑚 : 𝑏𝑏𝑏𝑏𝑏𝑏𝑜𝑤_𝑏\n💜💙❤🧡💚💛🖤'], dtype=object)

In [131]:
# 내 명시적인 동의 없이 트윗을 사용하지 마세요, 인용하지 마십시오, 허락을 🚫 무단사용 및 도용금지 and roots
kw ='무단사용 및 도용금지'#['DOES NOT',"DON'T USE",'research','credit','repost','scrap','media','steal','publication','permission','approve','approval','permit','collect', cite, quote]
twts = pd.Series(twts)
sub_twt = twts.loc[(twts.str.contains(kw, case=False,regex=False))]
for t in sub_twt:
    print(t+'\n')
    

7명 모두가 소듕한 아미입니다💜                                                                    주로 #어썸_작업 | 좋은날에 #어썸_이벵 |
후기는 #어썸한후기 | 무단사용 및 도용금지🙅‍♀️ | 나눔물품 금전거래 금지🚫

7명 모두가 소듕한 아미입니다💜                                                                    주로 #어썸_작업 | 좋은날에 #어썸_이벵 |
후기는 #어썸한후기 | 무단사용 및 도용금지🙅‍♀️ | 나눔물품 금전거래 금지🚫

7명 모두가 소듕한 아미입니다💜                                                                    주로 #어썸_작업 | 좋은날에 #어썸_이벵 |
후기는 #어썸한후기 | 무단사용 및 도용금지🙅‍♀️ | 나눔물품 금전거래 금지🚫



SWZ_D1

Permission mentions: (Steal too)
 mostly visual artists requesting people not repost their work without permission (note the Twitter API does not retain embedded media, only text with hyperlinks back to where twitter is hosting uploaded images and videos. so no violations there.)
 Also focus on reposting and crediting. We are not reposting, nore sharing the specific materials they are requesting credit for.

Cite mentions 1
"🚫 please do not cite my tweets w/o my express consent"
we aren't citing.

lol found my own bio in here. 

Research 1:
💥This acct DOES NOT consent to being used for research purposes 💥


🎨🖌Artist | 다시 게시하지 마십시오🚫❗❗


In [130]:
# kw
df_bios = df_alltwt.loc[df_alltwt['user_description'].notna(),:].copy()
twts = df_bios['user_description']
sub_twt = df_bios.loc[(twts.str.contains(kw, case=False,regex=False)),:]
# print(sub_twt['user_id'].unique())
# sub_twt

PTD_ON

"DON'T USE MY Tweets/screenshot them without MY express consent!"

More instances of "DON'T USE" and consent. part of the structure of this dataset. 107 of 277925 from 12 accounts. So do we cut them? sure.

one instance in coded data, but tagged as unrelated and lost so not in the published set. 

Is this a complete search? No keyword use is not super reliable, and search was limited crosslinguistically. However, the low number of these cases caught indicates that these concerns were not particularly concentrated in these datasets. 


# Covid traces



In [201]:
i = 0
df_alltwt=pd.read_csv(Concerts.loc[i,'data_loc'] + Concerts.loc[i,'twt_dbs'])
print(df_alltwt.columns)
twts = df_alltwt['tweet']
twts[:5]


Index(['id', 'created_at', 'tweet', 'source', 'language', 'user_id',
       'user_screen_name', 'user_name', 'user_description', 'user_language',
       'user_location', 'user_created_at', 'user_followers_count',
       'user_friends_count', 'user_statuses_count', 'user_favorites_count',
       'user_verified', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'in_reply_to_user_screen_name', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_user_screen_name',
       'retweeted_status_user_name', 'retweeted_status_user_description',
       'retweeted_status_user_friends_count',
       'retweeted_status_user_statuses_count',
       'retweeted_status_user_followers_count',
       'retweeted_status_retweet_count', 'retweeted_status_favorite_count',
       'retweeted_status_reply_count', 'quoted_status_id',
       'quoted_status_user_id', 'quoted_status_user_screen_name',
       'quoted_status_user_name', 'quoted_status_user_description',
       'quoted_status

0    RT @MygNamm01: 🎉แจกคอน Muster​ BTS2021​\n✨เพื่...
1    準備万端🥲🪐💜💜💜\n久しぶりのライブ楽しみ〜〜❕🌈\n\n#SOWOOZOO https:...
2    RT @mxnt_JK: แจกอีกค่ะ แจก 50 ฿ \n\n ⊹🐻. 𖧷 🧺 ꕀ...
3    [#SOWOOZOO]\n\nAujourd'hui aura lieu le 1er co...
4    RT @MoonJKV7: แจกคอน BTS MUSTER SOWOOZOO\nHD m...
Name: tweet, dtype: object

In [207]:
kws = ['remote', 'virtual', 'pandemic', 'covid', 'lockdown', 'restriction']
kw = kws[0]
print(kw)
twts = pd.Series(twts)
sub_twt = twts.loc[(twts.str.contains(kw, case=False,regex=False))]
print(len(sub_twt))
sub_twt = sub_twt.unique()
print(len(sub_twt))

remote
1
1


In [200]:
for t in sub_twt:
    print(t+'\n')

RT @ayshardzn: We have detected Omicron variant in Malaysia from a traveller arriving through KLIA. The person arrived on 19th Nov from South Africa via Singapore. This was before WHO announced Omicron &amp; before we imposed travel restriction on 8 countries.

Apa sial org tu masuk sini kenapa tak tutup border bodoh nya kerajaan sial QT @ayshardzn: We have detected Omicron variant in Malaysia from a traveller arriving through KLIA. The person arrived on 19th Nov from South Africa via Singapore. This was before WHO announced Omicron &amp; before we imposed travel restriction on 8 countries.

RT @NewsBFM: Malaysia has detected its first case of the Omicron variant. 

Health Minister Khairy Jamaluddin says the case involved a foreigner, aged 19, who arrived from South Africa through Singapore on November 19. https://t.co/QheuyVpNwQ QT @NewsBFM: 1. The new Covid-19 variant detected in Africa has been named "Omicron".

The WHO has classified it as a variant of concern but warns countries a